# Branch Gas chambers (Jonathan D. Müller & Itay Oz)

Loads the raw data of Yakir Preisler and Itay Oz's branch chambers, measured in Yatir, and applies corrections

In [ ]:
import os
import glob
import pandas as pd
import numpy as np

In [ ]:
# Initialization parameters

# input
project_path = './'
project_path_chambers = project_path + '01_raw_chamber_data/branch chambers/'
project_path_par      = project_path + '01_raw_chamber_data/PAR/'
project_path_additional = project_path + '00_additional_data/'
leaf_area_fn      = 'leaf_area.xlsx'
chamber_status_fn = 'chambers_logbook.xlsx'
spikes_fn         = 'spikes.csv'
par_logbook_fn    = 'PAR_sensors_logbook.xlsx'

# output
project_path_results = project_path + '02_preprocessed_chamber_data/branch_chambers_automatic/'
output_fn = 'chambers'

# List of months to process
# - If empty, all available data is processed
# - Otherwise, specify using a string of year-month, e.g. '2019-05', '2019-06'
#month_list = [ '2020-07', '2020-08', '2020-09', '2020-10', '2020-11' ]
#month_list = [  ] # Everything
month_list = [ '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06' ]

# Pump cleaning parameters: Values different from these will be deleted
min_abs_flow = 6       # Minimum allowed pump flow
max_delta_flow = 4     # Maximum difference between ambient and sample pumps
max_stddev_flow = 0.5  # Maximum standard deviation within pump flows

# Leaf area parameters
proj_la_factor = np.pi + 2 # Factor to convert total leaf area to projected leaf area
needle_diameter_irr = 0.82 # diameter [mm] of needles in the irrigation plot
needle_diameter_ctr = 0.89 # diameter [mm] of needles in the control plot
needle_diameter_LA  = 0.85 # reference "wrong" diameter [mm] of needles used in the input leaf area data
active_la_slope     = 35.21 # Active LA depends on mutual shading of twigs. Empirical value. Set to 1.00 if entire LA is active
active_la_intercept =  1.33 # Active LA depends on mutual shading of twigs. Empirical value. Set to 0.00 if entire LA is active

# Other parameters
dew_threshold = 98     # Percentage of RH when dew starts to form
par_threshold = 50     # Threshold by which to determine whether to keep PAR, positive or negative

# Calibration with chamber 8
h2o_calibration_threshold = 0.05 # The standard deviation allowed in chamber 8 measurements
co2_calibration_threshold = 0.5

# Amount on light penetrating the chamber walls compared to outside, in PAR range
chamber_attenuation_coefficient = 0.8783

# Factor of stomata on different sides of leaves, i.e. 1 for pine needles
K = 1

# When to remove the 2 chambers
cos_con_start = pd.to_datetime('2020-07-16 19:00', format='%Y-%m-%d %H:%M', utc=True) # Chamber 4
cos_irr_start = pd.to_datetime('2020-07-04 19:00', format='%Y-%m-%d %H:%M', utc=True) # Chamber 16

## Load additionally needed data
- Leaf area logbook
- Chambers status (working or not)
- Spikes
- Status of PAR sensors
- Special COS chambers, installed starting on 4/7/2020

In [ ]:
# Read additional data
#----------------------

# Read leaf area (la)
la = pd.read_excel(project_path_additional + leaf_area_fn, sheet_name='Total summary')
# Convert dates
la['timestamp'] = pd.to_datetime( la.year.map(str) + "-" + la.month.map(str), format='%Y.0-%m.0', utc=True, errors='coerce')
la = la[la.year.notnull()]
#display(la)

# Read chamber status (cs)
cs = pd.read_excel(project_path_additional + chamber_status_fn, usecols=list(range(0,19)))
# Convert dates
cs['timestamp'] = pd.to_datetime( cs.year.map(str) + "-" + cs.month.map(str) + "-" + cs.day.map(str), format='%Y-%m-%d', utc=True, errors='coerce')
# Filter out errors (too many days in a month)
cs = cs[cs.timestamp.notnull()]
#display(cs)

# Read spikes that were previously identified. Used to remove them
spikes = pd.read_csv(project_path_additional + spikes_fn)
spikes['timestamp'] = pd.to_datetime( spikes.timestamp, format='%d-%m-%Y %H:%M', utc=True, errors='coerce')

# Read PAR sensor installation and information
par_log = pd.read_excel(project_path_additional + par_logbook_fn)
par_log.rename(columns={'cham no.':'chamber'}, inplace=True)
par_log.rename(columns={'MUX port ':'port'}, inplace=True)
#display(par_log)

# Full processing pipeline

## Functions

In [ ]:
# Generic: Unit converstions, energy, etc.
#-----------------------------------------

# Calculate the dewpoint temperature in C
def calculate_dewpointC(T_C, RH):
    dp = 243.04*(np.log(RH/100)+((17.625*T_C)/(243.04+T_C)))/(17.625-log(RH/100)-((17.625*T_C)/(243.04+T_C)))
    return(dp)

# Calculate saturation vapour pressure from pressure and temperature
# - 2 methods are available. Jones uses air pressure, Campbell & Norman do not
def calculate_es(T_C, P_Pa):
    # Jones p.348 (appendix 4)
    #es = (1.00072+(10**(-7)*P_Pa*(0.032+5.9*10**(-6)*T_C**2))) * (611.21*np.exp( (18.678-(T_C/234.5))*T_C/(257.14+T_C) ))

    # Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    # Campbell & Norman (1998)
    T_K = T_C + 273.15
    es = T_K**(-8.2) * np.exp(77.345 + 0.0057*T_K - 7235 * T_K**(-1))
    return(es)

# Calculates VPD from different environmental variables
def calculate_VPD(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    e = h2o_mol_mol * P_Pa         # Water vapor partial pressure (Pa)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    VPD = es - e                   # VPD (Pa)
    return(VPD)

# Converts water concentration [mmol mol] to RH [%]
def convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]
    
    #es = calculate_es(T_C, P_Pa)
    #RH <- 0.263*P_Pa*((h2o_mmol_mol*18.02/28.97)/1000)*np.exp(17.67*(T_C)/(T_K-29.65))**(-1)
    #RH = 100 if (RH > 100) else RH
    #RH = np.nan if (RH < 5) else RH

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    e = h2o_mol_mol * P_Pa         # Water vapor partial pressure (Pa)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    RH = e/es * 100                # RH (%)
    return(RH)

# Density of dry air
# - https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_rho_dry_air(T_C, h2o_mmol_mol, P_Pa):
    # Constants
    R_dry_air = 287.058     # [J kg-1 K-1] Specific gas const dry air
    T_K = T_C + 273.15
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    
    rho_dry_air = P_d / (R_dry_air * T_K) # Density of dry air (use for approximation)
    return(rho_dry_air)

# Density of moist air
def calculate_rho_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mol_mol * M_h2o / v_a # Ambient water vapor mass density (kg m-3)

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    return(rho_air)

# Dry air heat capacity at constant pressure
# cp_d in [J kg-1 K-1]
 # https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_dry_air(T_C):
    cp = 1005 + ((T_C + 23.12)**2)/3364
    return(cp)

# Specific heat capacity of moist air at constant pressure
# cp_m in [J kg-1 K-1]
# https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # RH
    RH = convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa)

    # Water vapor heat capacity at constant pressure (cp_h2o, J kg-1 K-1)
    cp_h2o = 1859 + 0.13*RH + (0.193 + 5.6*10**(-3) * RH)*T_C + (10**(-3) + 5 * 10**(-5)*RH)*T_C**2

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mol_mol * M_h2o / v_a # Ambient water vapor mass density (kg m-3)

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    # Specific humidity (Q, kg kg-1) 
    Q = rho_h2o / rho_air

    # cp_moist
    cp = calculate_cp_dry_air(T_C) * (1-Q) + cp_h2o * Q
    return(cp)

def calculate_flux(T_C, P_Pa, airflow_lpm, mol_frac_in, mol_frac_out, area_m2):
    # General flux function for any gas
    
    # Temperature in K
    T_K = T_C + 273.15

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = mol_frac_out * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
        
    # The delta of the mole fraction of the gas coming in and going out of a chamber
    delta_mol_frac = mol_frac_out - mol_frac_in
    
    # Calculate the flow of the measured gas
    gas_flow = airflow_mol_s * delta_mol_frac
    
    # Calculated flux
    flux = gas_flow / area_m2

    return(flux)

def calculate_h2o_flux_li6400(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa), of chamber air (measurement of LPM is of chamber air)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (v_d, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (v_a, m3mol-1)
    
    # Convert airflow from LPM to [mol s-1]
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       This leads to eq. 1-3 changing to sE = uo*wo - (uo - sE)*we
    #       Finally, eq. 1-4 becomes E = uo*(wo - we) / (s*(1 - we))
    #       Where uo = airflow_mol_s
    #             wo = h2o_mol_mol_chamber (outgoing mol fraction)
    #             we = h2o_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
      
    # LI-6400 manual, page 1-7
    h2o_flux = (airflow_mol_s * (h2o_mol_mol_chamber - h2o_mol_mol_ambient)) / (area_m2 * (1 - h2o_mol_mol_ambient))
    
    return(h2o_flux) # mol.m-2.s-1
                                              
def calculate_co2_flux_li6400(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    co2_mol_mol_ambient = co2_umol_mol_ambient * 10**(-6)
    co2_mol_mol_chamber = co2_umol_mol_chamber * 10**(-6)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa), of chamber air (measurement of LPM is of chamber air)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
    
    # H2O transpiration flux [mol.m-2.s-1]
    h2o_flux_mol_m2_s = calculate_h2o_flux_li6400(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2)
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       The LI-6400 uses assimilation, which is a positive flux. Previously, we used negative numbers due to CO2 uptake
    #       The sign should reflect the direction of the flux, i.e. emission is positive, and uptake negative, so assimilation should be negative
    #       The following calculations will therefore change eq. 1-11 (p. 1-9) to sa = uo*co - ue*ce
    #       This leads to eq. 1-12 (p. 1-9) changing to sa = uo*co - (uo - sE)*ce
    #       Finally, eq. 1-13 becomes a = uo*(co-ce) / s + E*ce
    #       Where uo = airflow_mol_s
    #             co = co2_mol_mol_chamber (outgoing mol fraction)
    #             ce = co2_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
    #             E  = water flux (calculated above)
    
    # LI-6400 manual, page 1-13 (p. 1-10)
    co2_flux = (airflow_mol_s * (co2_mol_mol_chamber - co2_mol_mol_ambient)) / area_m2 + h2o_flux_mol_m2_s * co2_mol_mol_ambient
    
    return(co2_flux) # mol.m-2.s-1

def calculate_leaf_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3) # water in [mol mol-1]
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3) # water in [mol mol-1]
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa), of chamber air (measurement of LPM is of chamber air)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
    
    # H2O transpiration flux [mol.m-2.s-1]
    h2o_flux_mol_m2_s = calculate_h2o_flux_li6400(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2)
    
    # Leaf values
    es_leaf = calculate_es(T_leaf_C, P_Pa) # leaf water content, based on leaf temperature
    h2o_mol_mol_leaf = es_leaf / P_Pa # leaf water to air molar ratio
    
    # LI-6400 manual, eq 1-7 (p. 1-8)
    total_conductance = h2o_flux_mol_m2_s * (1 - (h2o_mol_mol_leaf + h2o_mol_mol_chamber)/2)/(h2o_mol_mol_leaf - h2o_mol_mol_chamber)
    
    return(total_conductance) # mol.m-2.s-1

def calculate_stomatal_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3) # water in [mol mol-1]
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3) # water in [mol mol-1]
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa), of chamber air (measurement of LPM is of chamber air)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3) # water in [mol mol-1]
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3) # water in [mol mol-1]
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa), of chamber air (measurement of LPM is of chamber air)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3 s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol s-1]
    
    # LI-6400 manual, eq 16-6 (p. 16-60)
    stomatal_conductance = (airflow_mol_s*(h2o_mol_mol_chamber - h2o_mol_mol_ambient)/area_m2) * (1/(h2o_mol_mol_leaf - h2o_mol_mol_chamber))
    
    return(stomatal_conductance)

def calculate_bl_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2, K):
    total_cond = calculate_leaf_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2)
    stoma_cond = calculate_stomatal_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2)
    
    # Factor of fraction of stomatal conductances of one side to the other of a leaf
    kf = (K**2+1)/(K+1)**2
    
    # Boundary layer conductance
    # LI-6400 manual, eq 1-9 (p. 1-9)
    bl_conductance = kf / (1/total_cond - 1/stoma_cond)
    return(bl_conductance)

# Leaf conductance to CO2
def calculate_leaf_co2_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2, K):
    bl_cond    = calculate_bl_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2, K)
    stoma_cond = calculate_stomatal_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2)
    
    # CO2
    # (Bonan 2019, eq. 12.5)
    co2_conductance = 1 / (1.4*bl_cond**(-1) + 1.6*stoma_cond**(-1))
    return(co2_conductance)

# Intercellular CO2 concentration
def calculate_leaf_co2_concentration(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2, K):
    # Unit conversion
    co2_mol_mol_ambient = co2_umol_mol_ambient * 10**(-6)
    # Leaf conductance to CO2
    co2_cond    = calculate_bl_h2o_conductance(T_C, T_leaf_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2, K)
    # Assimilation CO2 flux
    An = calculate_co2_flux_li6400(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, co2_umol_mol_ambient, co2_umol_mol_chamber, airflow_lpm, area_m2)
    
    # CO2 intercellular concentration
    # (Bonan 2019, eq. 12.4)
    co2_conc = co2_mol_mol_ambient - An / co2_cond
    return(co2_conc)

# Energy calculations
#--------------------

# Enthalpy of vaporisation/condensation Lwater
# Heat released due to vaporisation/condensation, used to convert [mmol.m-2.s-1] to [W.m-2]
# https://en.wikipedia.org/wiki/Latent_heat#Specific_latent_heat_for_condensation_of_water_in_clouds
def calculate_enthalpy_vaporisation(T_C):
    # If transpiration <0 & leaf = wet, use this for LE (negative) ????????????
    
    # Empirical: https://www.engineeringtoolbox.com/water-properties-d_1573.html
    # 2°C  = 44970 J/mol at saturation pressure
    # 50°C = 42911 J/mol at saturation pressure
    
    # Constants
    molar_weight_h2o = 18.01528 # [g mol-1]
    
    # From -25°C to +40°C, Lwater is approximated as [J g-1]
    Lwater_J_g = (2500.8 - 2.36*T_C + 0.0016*T_C**2 - 0.00006*T_C**3)
    
    # Conversion to [J.mol-1]
    Lwater = Lwater_J_g * molar_weight_h2o
    
    return(Lwater)

# Calculates latent heat based on transpiration
def calculate_latent_heat(T_C, h2o_flux_mmol_m2_s, P_Pa):
    # Convert units to [mol m-2 s-1]
    h2o_flux_mol_m2_s = h2o_flux_mmol_m2_s * 10**(-3)
    # Enthalpy of vaporisation/condensation Lwater
    Lwater = calculate_enthalpy_vaporisation(T_C)
    # Latent heat
    LE = Lwater * h2o_flux_mol_m2_s
    return(LE)

# Calculates the sensible heat flux
# https://www.engineeringtoolbox.com/sensible-heat-flow-d_702.html
def calculate_sensible_heat(T_chamber, T_ambient, h2o_mmol_mol, airflow_lpm, P_Pa, leaf_area):    
    # change in temperature due to the flux
    deltaT = T_chamber - T_ambient
    # Heat capacity of moist air cp
    cp = calculate_cp_moist_air(T_ambient, h2o_mmol_mol, P_Pa)
    # Density of moist air rho
    rho = calculate_rho_moist_air(T_ambient, h2o_mmol_mol, P_Pa)
    # Convert airflow from lpm to [m3.s-1]
    q = airflow_lpm / 1000 / 60
    # Sensible heat H [W.m-2] (Note: Leaf area is in m-2)
    H = rho * cp * q * deltaT / leaf_area
    return(H)

In [ ]:
# Functions called by other functions
#-------------------------------------

# Assign block IDs for grouping and averaging in calculations
def create_block_id(df):
    blockID = 0
    blockIDs = []
    x_old = df["chamber"].tolist()[0]

    for x in df["chamber"].tolist():
        # If same chamber, give same blockID
        if x == x_old:
            blockIDs.append(blockID)
            continue
        # If switch to new chamber, give new blockID
        else:
            blockID += 1
            blockIDs.append(blockID)
        x_old = x

    # Add block ID
    df.loc[:,'chamber_block'] = blockIDs

# Finds the leaf areas for each chamber for the current day
def find_leaf_area(df, leaf_area):
    # Determine the current day
    day = df['timestamp'].dt.normalize().unique()[0]
    # Create the column filter. Chambers 8 & 16 don't have leaves, so no values
    col_filter = [col for col in leaf_area if col.startswith('chamber')]
    # Select relevant columns
    temp = leaf_area[(leaf_area.year == day.year) & (leaf_area.month == day.month)][col_filter]
    if(len(temp) != 1):
        print("Error: No leaf area available")
    # Convert from wide format to proper usable df in long format
    temp2 = temp.stack().reset_index()
    temp2['chamber'] = temp2['level_1'].map(lambda x: x.strip('chamber ')).astype(int)
    temp2.rename(columns={0:'leaf.area.m2'}, inplace=True)
    temp2.drop('level_0', axis=1, inplace=True)
    temp2.drop('level_1', axis=1, inplace=True)
    # Correct leaf area by needle diameter
    temp2.loc[temp2['chamber'] <= 7, 'leaf.area.m2'] = temp2.loc[temp2['chamber'] <= 7, 'leaf.area.m2'] * (needle_diameter_irr/needle_diameter_LA)
    temp2.loc[(temp2['chamber'] >= 9) & (temp2['chamber'] <= 15), 'leaf.area.m2'] = temp2.loc[(temp2['chamber'] >= 9) & (temp2['chamber'] <= 15), 'leaf.area.m2'] * (needle_diameter_ctr/needle_diameter_LA)
    # Add dummy factors for empty chambers 8 & 16
    temp2 = pd.concat( [pd.DataFrame({"leaf.area.m2":[1, 1], "chamber":[8, 16]}),temp2], ignore_index=True)
    
    return(temp2)

# Read a CR1000 PAR input file
def read_par_input_file(input_fn):
    df = pd.read_csv(input_fn,skiprows=[0,2,3], na_values=["NAN"])
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4], na_values=["NAN"])
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")
    return(df)

# Finds the relevant file name of a PAR file for a given date
def find_par_file(file_list, this_day):
    # We need the day ID as a string
    current_day = str(this_day)
    # Convert current day string to equivalent format as datalogger file name
    current_day = current_day[0:4] + '_' + current_day[4:6] + '_' + current_day[6:8]
    # Make a list of all relevant file names
    fn = [i for i in file_list if current_day in i] 
    return(fn)

# Read the PAR log, and extract the relevant data 
def read_apogee_par(file_list, this_day):
    # Find the PAR file name of the current day
    par_fn = find_par_file(file_list, this_day)[0]
    # Read PAR file
    par_raw_df = read_par_input_file(par_fn)
    
    # Create empty PAR df
    par_df = par_raw_df[['timestamp']].copy()
    for i in range(1,17):
        par_df['par.ch' + str(i) + '.umol_m2_s'] = np.nan
    
    # Find relevant log entries
    previous_log_date = par_log['date'].unique()[0]
    log_entry_date = 0
    for i in par_log['date'].unique():
        current_date = pd.to_datetime(str(this_day), format='%Y%m%d')
        # Find the entry that's in between the current date and the previous
        if((current_date > previous_log_date) and (current_date < i)):
            log_entry_date = previous_log_date
        # If the current date is larger than the last entry, keep the last entry
        if(current_date > par_log['date'].unique()[-1]):
            log_entry_date = i
        # If the entry is on a measurement day, make it 0 so that NAs are filled in
        if(current_date == i):
            log_entry_date = 0
        previous_log_date = i

    # Find the relevant chamber and port, and fill the df
    for i in par_log.loc[(par_log['date'] == log_entry_date) & (par_log['status'] == 'good'),'chamber']:
        port = par_log.loc[(par_log['date'] == log_entry_date) & (par_log['chamber'] == i),'port'].values[0]
        par_df['par.ch' + str(i) + '.umol_m2_s'] = par_raw_df['par_Avg(' + str(port) + ')']
    
    # The sensor values need to be multiplied by 100 to convert mV to umol.m-2.s-1
    for i in range(1,17):
        par_df['par.ch' + str(i) + '.umol_m2_s'] = par_df['par.ch' + str(i) + '.umol_m2_s'] * 100
    
    return(par_df)

def classify_chamber_plots(temp):
    # Add colummns with the plot name, used for averaging
    temp = temp.copy()
    
    # Chambers 1-7 irrigation
    # Chambers 9-15 control
    # Chamber 8 is ambient
    # Chamber 16 is soil,
    #          then outside the ambient chamber 8 from '2020-05-21 15:00',
    
    temp['plot'] = np.nan
    temp.loc[(temp['chamber'] <= 7), 'plot'] = "irrigation"
    temp.loc[(temp['chamber'] == 8), 'plot'] = "ambient"
    temp.loc[(temp['chamber'] >= 9) & (temp['chamber'] <= 15), 'plot'] = "control"
    temp.loc[(temp['chamber'] == 16), 'plot'] = "soil"
    temp.loc[(temp['chamber'] == 16) & (temp['timestamp'] >= '2020-05-21 15:00'), 'plot'] = "ambient out"
    
    return(temp)

# Makes hourly means per plot
def make_1h_means(df):
    # Classify  chambers
    df = classify_chamber_plots(df)
    
    # Detect dew formation
    df['dew'] = np.nan
    df.loc[df['rh.air.chamber'] >= dew_threshold, 'dew'] = True
    df.loc[df['rh.air.chamber'] <  dew_threshold, 'dew'] = False
    
    # Make the means
    df_1h = df.groupby(['plot','dew']).resample('1h', on='timestamp', closed='right').mean()
    # Remove weird indices
    df_1h.reset_index(inplace=True) # Makes the plot into a column
    
    # Add a count of occurences of chambers, differentiating between ['flux.h2o.mmol_m2_s','flux.co2.umol_m2_s']
    def count_non_nan(data):
        counted = sum(~np.isnan(data))
        return(counted)
    temp_1min = make_1min_means(df)
    df_1h_n = temp_1min.groupby(['plot','dew']).resample('1h', on='timestamp', closed='right').agg({'flux.h2o.mmol_m2_s': count_non_nan,'flux.co2.umol_m2_s': count_non_nan})
    df_1h_n = df_1h_n.astype(int)
    df_1h_n.reset_index(inplace=True)
    df_1h_n.rename(columns={'flux.h2o.mmol_m2_s':'flux.h2o.n'}, inplace=True)
    df_1h_n.rename(columns={'flux.co2.umol_m2_s':'flux.co2.n'}, inplace=True)
    
    # Merge the counts
    df_1h = df_1h.merge(df_1h_n, on=['timestamp','plot','dew'], how="left")
    # Sort by time
    df_1h.sort_values(["timestamp", "plot","dew"], inplace=True)
    # Move timestamp column to the front
    col = df_1h.pop('timestamp') # df.pop('F') # if you want it removed
    df_1h.insert(0, col.name, col, allow_duplicates=True)
    return(df_1h)

# Makes means per chamber and time
def make_1min_means(df):
    # Makes 1min means for each chamber block
    create_block_id(df)
    
    # Make the means
    df_1min = df.groupby('chamber_block').resample('1min', on='timestamp', closed='right').mean()
    df_1min.drop('chamber_block', axis=1, inplace=True)
    df_1min.drop('Counter', axis=1, inplace=True)
    #grouped = df.groupby("chamber_block")
    #for x in grouped:
    #    time = pd.Timestamp(np.mean(x[1]["timestamp"])).round('1M')
    #print(time)  
    
    # Remove weird indices
    df_1min.reset_index(inplace=True) # Makes the plot into a column
    
    # Classify  chambers
    df_1min = classify_chamber_plots(df_1min)
    
    # Detect dew formation
    df_1min['dew'] = np.nan
    df_1min.loc[df_1min['rh.air.chamber'] >= dew_threshold, 'dew'] = True
    df_1min.loc[df_1min['rh.air.chamber'] <  dew_threshold, 'dew'] = False
    
    # Sort by time
    df_1min.sort_values(["timestamp", "chamber", "plot","dew"], inplace=True)
    
    # Move timestamp column to the front
    col = df_1min.pop('dew')
    df_1min.insert(0, col.name, col, allow_duplicates=True)
    col = df_1min.pop('plot')
    df_1min.insert(0, col.name, col, allow_duplicates=True)
    col = df_1min.pop('chamber')
    df_1min.insert(0, col.name, col, allow_duplicates=True)
    col = df_1min.pop('timestamp')
    df_1min.insert(0, col.name, col, allow_duplicates=True)
    # Remove now obsolete column
    df_1min.drop('chamber_block', axis=1, inplace=True)
    #df_1min.drop('index', axis=1, inplace=True)
    
    return(df_1min)

In [ ]:
# Main functions
#---------------

# Delete all files in a folder.
# - Used to remove 1h and 1min monthly files before running to prevent appending to existing files
def empty_dir_all(directory):
    files = glob.glob(directory + '*')
    for f in files:
        os.remove(f)
    pass

# Delete all relevant files in a folder.
# - Used to remove 1h and 1min monthly files before running to prevent appending to existing files
def empty_dir(directory):
    files = glob.glob(directory + '*')
    for f in files:
        month_id = f[-10:-6] + '-' + f[-6:-4]
        if(month_id in month_list):
            print('Remove ' + f)
            os.remove(f)
        if(not month_list):
            print('Remove ' + f)
            os.remove(f)
    pass

# Read a CR1000 input file
# - Reads all data in the file
def read_input_file(input_fn):
    df = pd.read_csv(input_fn,skiprows=[0,2,3,4,5], na_values=["NAN"])
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5,6], na_values=["NAN"])
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read a CR1000 input file by lines
# - Reads only some initial lines
def read_input_file_by_lines(input_fn, lines):
    df = pd.read_csv(input_fn,skiprows=[0,2,3,4], na_values=["NAN"],nrows=lines)
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5], na_values=["NAN"],nrows=lines)
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)
    
# Changes column names and drops unneeded ones
def fix_column_names(temp):
    temp = temp.copy()
    
    # Remove obsolete columns
    temp.drop('RECORD', axis=1, inplace=True)
    temp.drop('batt_volt', axis=1, inplace=True)
    temp.drop('PTemp', axis=1, inplace=True)
    temp.drop('Temp_6262', axis=1, inplace=True)
    temp.drop('Volt_AM25(1)', axis=1, inplace=True)
    temp.drop('Volt_AM25(2)', axis=1, inplace=True)
    temp.drop('Volt_AM25(3)', axis=1, inplace=True)
    temp.drop('Pump_Status', axis=1, inplace=True)
    temp.drop("ChamberStatus", axis=1, inplace=True)
    
    # Rename PAR
    temp.rename(columns={'RadKipZonen':'par.ambient.umol_m2_s'}, inplace=True)
    temp.rename(columns={'PAR':'par.current.chamber.umol_m2_s'}, inplace=True)
    
    # Replace ambient air T
    temp.drop('Ambient_TempAir', axis=1, inplace=True)
    #temp.drop('Tc_AM25', axis=1, inplace=True) # Since 2020-05-21 at 15:00 or so this is the wall of the ambient chamber 8
    temp.rename(columns={'Tc_AM25':'temp.wall.ambient.c'}, inplace=True)
    temp.rename(columns={'Tc(8)':'temp.air.ambient.c'}, inplace=True)
    temp.rename(columns={'Tc(16)':'temp.air.controlbox.c'}, inplace=True) # Since 2020-05-21 at 15:00 this is the air outsidee ambient chamber 8
    
    temp.rename(columns={'Tc(1)':'temp.air.ch1.c'}, inplace=True)
    temp.rename(columns={'Tc(2)':'temp.air.ch2.c'}, inplace=True)
    temp.rename(columns={'Tc(3)':'temp.air.ch3.c'}, inplace=True)
    temp.rename(columns={'Tc(4)':'temp.air.ch4.c'}, inplace=True)
    temp.rename(columns={'Tc(5)':'temp.air.ch5.c'}, inplace=True)
    temp.rename(columns={'Tc(6)':'temp.air.ch6.c'}, inplace=True)
    temp.rename(columns={'Tc(7)':'temp.air.ch7.c'}, inplace=True)
    temp.rename(columns={'Tc(9)':'temp.air.ch9.c'}, inplace=True)
    temp.rename(columns={'Tc(10)':'temp.air.ch10.c'}, inplace=True)
    temp.rename(columns={'Tc(11)':'temp.air.ch11.c'}, inplace=True)
    temp.rename(columns={'Tc(12)':'temp.air.ch12.c'}, inplace=True)
    temp.rename(columns={'Tc(13)':'temp.air.ch13.c'}, inplace=True)
    temp.rename(columns={'Tc(14)':'temp.air.ch14.c'}, inplace=True)
    temp.rename(columns={'Tc(15)':'temp.air.ch15.c'}, inplace=True)
    
    # Fix thermocouple naming after they were moved
    mask_moving_start = (temp['timestamp'] < '2020-05-21 09:00')
    mask_moving_end   = (temp['timestamp'] < '2020-05-21 15:00')
    temp.loc[mask_moving_end, 'temp.wall.ambient.c'] = np.nan # New measurement, nan before moving was finished
    # The thermocouple of the controlbox was moved to the outside of the ambient chamber (8)
    temp['temp.air.ambient.out.c'] = temp['temp.air.controlbox.c']
    temp.loc[mask_moving_end, 'temp.air.ambient.out.c'] = np.nan # before moving finished, ambient outside didn't exist
    temp.loc[~mask_moving_start, 'temp.air.controlbox.c'] = np.nan # after moving started, the temperature control box temperature is not valid any more
    
    # Rename columns
    temp.rename(columns={'H2o_6262_mmol_mol':'h2o.ambient.mmol_mol'}, inplace=True)
    temp.rename(columns={'H2o_7000_mmol_mol':'h2o.chamber.mmol_mol'}, inplace=True)
    
    temp.rename(columns={'Co2_6262_micmol_mol':'co2.ambient.umol_mol'}, inplace=True)
    temp.rename(columns={'Co2_7000_micmol_mol':'co2.chamber.umol_mol'}, inplace=True)
    
    temp.rename(columns={'AirFlow_Sam':'pump.flow.chamber.lpm'}, inplace=True)
    temp.rename(columns={'AirFlow_Amb':'pump.flow.ambient.lpm'}, inplace=True)
    
    temp.rename(columns={'Temp_7000':'temp.irga.c'}, inplace=True)
    temp.rename(columns={'Prees_7000':'press.irga.kpa'}, inplace=True)
    
    temp.rename(columns={'Ambient_RH':'rh.air.ambient'}, inplace=True)
    temp.rename(columns={'VPD_Ambient':'vpd.air.ambient.kpa'}, inplace=True)
    temp.rename(columns={'VPD_Chamber':'vpd.air.chamber.kpa'}, inplace=True)
    
    temp.rename(columns={'H2o_Flux':'flux.h2o.mmol_m2_s'}, inplace=True)
    temp.rename(columns={'Co2_Flux':'flux.co2.umol_m2_s'}, inplace=True)

    temp.rename(columns={'ChamberON':'chamber'}, inplace=True)
    
    # Drop duplicates
    temp.drop_duplicates(subset = 'timestamp', inplace=True)
    
    # Drop false chamber 0
    temp.drop(temp[temp['chamber'] == 0].index, axis=0, inplace=True)
    
    # Sort, in case it's not yet the case
    temp.sort_values('timestamp', inplace=True)
    
    return(temp)

def fix_false_values(temp):
    temp = temp.copy()
    
    # Before 2018-12-14, the LI-7000 values were the difference from ambient, not absolute values. Convert to absolute
    temp.loc[temp['timestamp'] < '2018-12-14 00:00','h2o.chamber.mmol_mol'] = temp.loc[temp['timestamp'] < '2018-12-14 00:00','h2o.ambient.mmol_mol'] + temp.loc[temp['timestamp'] < '2018-12-14 00:00','h2o.chamber.mmol_mol']
    temp.loc[temp['timestamp'] < '2018-12-14 00:00','co2.chamber.umol_mol'] = temp.loc[temp['timestamp'] < '2018-12-14 00:00','co2.ambient.umol_mol'] + temp.loc[temp['timestamp'] < '2018-12-14 00:00','co2.chamber.umol_mol']
    
    # The gas tubes of chambers 13 & 16 were exchanged for a period: 14.5.2017 – 22 march 2018
    #temp['temp'] = temp['h2o.chamber.mmol_mol']
    temp.loc[(temp['chamber'] == 13) & (temp['timestamp'] > '2017-05-14 23:59') & (temp['timestamp'] < '2018-03-22 00:00'),'chamber'] = 0 # change temporarily to avoid mixing
    temp.loc[(temp['chamber'] == 16) & (temp['timestamp'] > '2017-05-14 23:59') & (temp['timestamp'] < '2018-03-22 00:00'),'chamber'] = 13
    temp.loc[(temp['chamber'] == 0)  & (temp['timestamp'] > '2017-05-14 23:59') & (temp['timestamp'] < '2018-03-22 00:00'),'chamber'] = 16
    
    # PAR is now incorrect for these two, because it was measured for 16 while the gase was measured for 13 and vice versa
    temp.loc[(temp['chamber'] == 13) & (temp['timestamp'] > '2017-05-14 23:59') & (temp['timestamp'] < '2018-03-22 00:00'),'par.current.chamber.umol_m2_s'] = np.nan
    temp.loc[(temp['chamber'] == 16) & (temp['timestamp'] > '2017-05-14 23:59') & (temp['timestamp'] < '2018-03-22 00:00'),'par.current.chamber.umol_m2_s'] = np.nan
    
    return(temp)

# Removes the first 4min of each day, when the system was autocalibrating
def remove_midnight_4min(temp):
    # Create the mask of first 4min of each day
    mask_first_4min = pd.DatetimeIndex(temp['timestamp']).indexer_between_time('0:00', '0:04')
    # Reset index, in case temp doesn't start with 0
    temp.reset_index(inplace=True)
    # Remove the relevant rows
    temp.drop(mask_first_4min, inplace=True)
    return(temp)
    
# Makes a column that has the current chamber air temperature
def current_chamber_air_t(temp):
    temp = temp.copy()
    temp['temp.air.current.chamber.c'] = np.nan
    for i in np.unique(temp['chamber'].astype(int)):
        if(i not in [8, 16]):
            temp.loc[temp['chamber'] == i, 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.air.ch' + str(i) + '.c']
            pass
        else:
            temp.loc[temp['chamber'] == i, 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.air.ambient.c']
    return(temp)

# Fixes PAR
def fix_PAR(temp):
    # Night-time fixes: Set Kipp & Zonen to 0 during the night
    temp.loc[temp['par.ambient.umol_m2_s'] < 10, 'par.ambient.umol_m2_s'] = 0
    
    # Delete whole day if night is bad, i.e. if the night is < -20 or > 20
    bad_chamber_par_list = []
    for x in temp.groupby("chamber"):
        if( (np.min(x[1]['par.ambient.umol_m2_s']) == 0) & (np.mean(x[1]['par.current.chamber.umol_m2_s']) < -par_threshold)):
            bad_chamber = np.min(x[1]['chamber'])
            bad_chamber_par_list.append(bad_chamber)
        if( (np.min(x[1]['par.ambient.umol_m2_s']) == 0) & (np.mean(x[1]['par.current.chamber.umol_m2_s']) >  par_threshold)):
            bad_chamber = np.min(x[1]['chamber'])
            bad_chamber_par_list.append(bad_chamber)
        pass
    bad_chamber_par_list = [int(i) for i in bad_chamber_par_list] # Convert to int, in case it isn't already
    print("    Remove bad PAR in chambers: ", bad_chamber_par_list)
    
    # Remove the data of the relevant chambers
    temp.loc[temp['chamber'].isin(bad_chamber_par_list), 'par.current.chamber.umol_m2_s'] = np.nan
    
    # Night-time fixes: Set to 0 during the night
    temp.loc[temp['par.current.chamber.umol_m2_s'] < 10, 'par.current.chamber.umol_m2_s'] = 0  # All photodiode values should be 0 when Kipp&Zonen is 0
    temp.loc[(temp['par.ambient.umol_m2_s'] == 0) & (temp['par.current.chamber.umol_m2_s'] > par_threshold), 'par.current.chamber.umol_m2_s'] = np.nan # large night-time values
    temp.loc[(temp['par.ambient.umol_m2_s'] == 0) & (temp['par.current.chamber.umol_m2_s'] > 0), 'par.current.chamber.umol_m2_s'] = 0       # photodiode non-zero at night
    
    # Daytime: When Kipp&Zonen > 0 and the photodiode is low, remove it
    temp.loc[(temp['par.ambient.umol_m2_s'] > 0) & (temp['par.current.chamber.umol_m2_s'] < 50), 'par.current.chamber.umol_m2_s'] = np.nan # remove small day-time values

    return(temp)

# Fixes RH
def fix_RH(temp):
    # The Vaisala RH sensor broke in early 2019. Remove newer data
    mask = (temp['timestamp'] > '2019-01-01')
    temp.loc[mask, 'rh.air.ambient'] = np.nan
    
    # In 2019, add RH calculated from ambient data
    temp['rh.air.ambient.from_irga'] = convert_mmol_RH(temp['temp.air.ambient.c'], temp['h2o.ambient.mmol_mol'], temp['press.irga.kpa']*1000)
    #temp['rh.air.chamber'] = convert_mmol_RH(temp['temp.air.current.chamber.c'], temp['h2o.chamber.mmol_mol'], temp['press.irga.kpa']*1000)
    temp['rh.air.chamber'] = convert_mmol_RH(temp['temp.air.current.chamber.c'], temp['h2o.chamber.mmol_mol'], temp['press.irga.kpa']*1000)
    return(temp)
    
# Adds the PAR data from the Apogee sensor to the main data
def add_apogee_PAR(temp, project_path_par, this_day):
    temp = temp.copy()
    
    # Date of installation of new PAR system: Apogee SQ-500-ss
    apogee_par_date = pd.to_datetime('2020-07-09', format='%Y-%m-%d')
    
    # The photodiodes are not connected any more after this date: erase and replace
    mask = (temp['timestamp'].dt.tz_localize(None) >= apogee_par_date)
    temp.loc[mask, 'par.current.chamber.umol_m2_s'] = np.nan
    
    # Make list of PAR file names
    par_fn_list = sorted(glob.glob(project_path_par + '*/*', recursive=True))
    # Read current PAR data (or create an empty df if necessary)
    if(len(find_par_file(par_fn_list, str(this_day))) == 0): # No such file exists, make an empty PAR df
        # create emty PAR df
        par_df = temp[['timestamp']].copy()
        for i in range(1,17):
            par_df['par.ch' + str(i) + '.umol_m2_s'] = np.nan
    else:
        par_df = read_apogee_par(par_fn_list, str(this_day))
        
    # Merge PAR data
    merged = temp.merge(par_df, on="timestamp", how="left")
    # Interpolate
    for i in range(1,17):
        merged['par.ch' + str(i) + '.umol_m2_s'].interpolate(method='linear', limit_direction='both', inplace=True)
    
    # Night-time corrections, by cycling through chambers: Set to 0 during the night
    for i in range(1,17):
        merged.loc[(merged['par.ch' + str(i) + '.umol_m2_s'] < 20), 'par.ch' + str(i) + '.umol_m2_s'] = 0 # All PAR values should be 0 at night. Small values are errors
        merged.loc[(merged['par.ambient.umol_m2_s'] == 0) & (merged['par.ch' + str(i) + '.umol_m2_s'] > par_threshold), 'par.ch' + str(i) + '.umol_m2_s'] = np.nan # Remove large night-time values
        merged.loc[(merged['par.ambient.umol_m2_s'] == 0) & (merged['par.ch' + str(i) + '.umol_m2_s'] > 0), 'par.ch' + str(i) + '.umol_m2_s'] = 0                  # Apogee non-zero at night
        
    # Make chamber list
    ch_list = merged.loc[merged['chamber'].notna(),'chamber'].unique().astype(int)
    # Copy the PAR data to PAR of current chamber column
    mask = (merged['timestamp'].dt.tz_localize(None) >= apogee_par_date)
    for i in ch_list:
        if(i not in [8]): # Copy data for all chambers, but not 8 (which is ambient)
            merged.loc[mask & (merged['chamber'] == i), 'par.current.chamber.umol_m2_s'] = merged.loc[mask & (merged['chamber'] == i), 'par.ch' + str(i) + '.umol_m2_s'] * chamber_attenuation_coefficient
        else: # If it is chamber 8, insert the ambient PAR measurement
            merged.loc[mask & (merged['chamber'] == i), 'par.current.chamber.umol_m2_s'] = merged.loc[mask & (merged['chamber'] == i), 'par.ambient.umol_m2_s']
    
    # Copy the PAR of current chamber column to each relevant chamber for the old data
    mask = (merged['timestamp'].dt.tz_localize(None) < apogee_par_date)
    for i in ch_list:
        if(i not in [8]): # Don't do anything for ambient measurements
            merged.loc[mask & (merged['chamber'] == i), 'par.ch' + str(i) + '.umol_m2_s'] = merged.loc[mask & (merged['chamber'] == i), 'par.current.chamber.umol_m2_s'] / chamber_attenuation_coefficient
        else: # If it is chamber 8, insert the ambient PAR measurement
            merged.loc[mask & (merged['chamber'] == i), 'par.ambient.umol_m2_s'] = merged.loc[mask & (merged['chamber'] == i), 'par.current.chamber.umol_m2_s']
    
    return(merged)


# Get the last 40seconds of each block
def get_last_30s(temp):
    # Create the block variable
    create_block_id(temp)
    selected = []
    for x in temp.groupby("chamber_block"):
        # Assemble timemasks
        # Get the final 40 seconds
        mask_final30s = (x[1]["timestamp"] > x[1]["timestamp"].tolist()[-1] + pd.Timedelta('-40 seconds'))
        if(np.unique(x[1]['chamber']) == 8):
            mask_final30s = (x[1]["timestamp"] > x[1]["timestamp"].tolist()[-1] + pd.Timedelta('-20 seconds'))

        # Discard final 10 seconds (optional)
        mask_final10s = (x[1]["timestamp"] < x[1]["timestamp"].tolist()[-1] + pd.Timedelta('-10 seconds'))

        # Union of masks is the desired data
        mask = mask_final30s #& mask_final10s
        
        # Check with the counter
        if(np.min(x[1][mask]["Counter"]) < 200):
            print("    Warning: Min. counter at ", np.min(x[1][mask]["Counter"]), " in chamber ", np.unique(x[1][mask]["chamber"])[0], " at ", np.min(x[1][mask]["timestamp"]))
            #selected.append(x[1][mask])
            pass
        else: # Prevents writing the current group if the data is out of range as a simple failsafe
            selected.append(x[1][mask])
            
    return pd.concat(selected)

# Simpler method to get the last 40seconds of each block
def keep_last_30s(temp):
    create_block_id(temp)
    return temp.loc[(temp['Counter'] >= (240-30)) & (temp['Counter'] <= 240)]

# Correct for fluctuations of the ambient measurement (chamber 8)
def remove_ambient_fluctuations(df):
    
    # Bad water list
    temp = df[df['chamber'] == 8].groupby('chamber_block').std()
    temp = temp.reset_index()
    temp['start'] = temp['chamber_block'].shift(1)
    temp['end']   = temp['chamber_block'].shift(-1)
    
    # Step 2
    # H2O
    bad_h2o_list = [ ]
    for i, bad_block in enumerate(temp.loc[temp['h2o.ambient.mmol_mol'].abs() > h2o_calibration_threshold, 'chamber_block']):
        start = temp.loc[temp['chamber_block'] == bad_block, 'start'].values[0] + 1
        end = temp.loc[temp['chamber_block'] == bad_block, 'end'].values[0] - 1
        if(start != start): # checks if value is nan
            start = np.min(df['chamber_block'].unique())
        if(end != end):
            end = np.max(df['chamber_block'].unique())
        bad_h2o_list = bad_h2o_list + list(range(int(start), int(end))) # Add list of bad data
    #print('    Bad H2O block list: ', bad_h2o_list)
    
    # CO2
    bad_co2_list = [ ]
    for i, bad_block in enumerate(temp.loc[temp['co2.ambient.umol_mol'].abs() > co2_calibration_threshold, 'chamber_block']):
        start = temp.loc[temp['chamber_block'] == bad_block, 'start'].values[0] + 1
        end = temp.loc[temp['chamber_block'] == bad_block, 'end'].values[0] - 1
        if(start != start): # checks if value is nan
            start = np.min(df['chamber_block'].unique())
        if(end != end):
            end = np.max(df['chamber_block'].unique())
        bad_co2_list = bad_co2_list + list(range(int(start), int(end))) # Add list of bad data
    #print('    Bad CO2 block list: ', bad_co2_list)
    
    # Step 1
    # H2O
    bad_h2o_list = [ ]
    for i, bad_block in enumerate(temp.loc[temp['h2o.ambient.mmol_mol'].abs() > h2o_calibration_threshold, 'chamber_block']):
        #start = temp.loc[temp['chamber_block'] == bad_block, 'start'].values[0] + 1
        #end = temp.loc[temp['chamber_block'] == bad_block, 'end'].values[0] - 1
        #if(start != start): # checks if value is nan
        #    start = np.min(df['chamber_block'].unique())
        #if(end != end):
        #    end = np.max(df['chamber_block'].unique())
        #bad_h2o_list = bad_h2o_list + list(range(int(start), int(end))) # Add list of bad data
        bad_h2o_list.append(bad_block)
    print('    Bad H2O block list: ', bad_h2o_list)
    
    # CO2
    bad_co2_list = [ ]
    for i, bad_block in enumerate(temp.loc[temp['co2.ambient.umol_mol'].abs() > co2_calibration_threshold, 'chamber_block']):
        #start = temp.loc[temp['chamber_block'] == bad_block, 'start'].values[0] + 1
        #end = temp.loc[temp['chamber_block'] == bad_block, 'end'].values[0] - 1
        #if(start != start): # checks if value is nan
        #    start = np.min(df['chamber_block'].unique())
        #if(end != end):
        #    end = np.max(df['chamber_block'].unique())
        #bad_co2_list = bad_co2_list + list(range(int(start), int(end))) # Add list of bad data
        bad_co2_list.append(bad_block)
    print('    Bad CO2 block list: ', bad_co2_list)
    
    debug_graph_fn = 'ch8_' + str(np.unique(df['dayid'])[0]) 
    debug_graph_fn2 = 'ch8_' + str(np.unique(df['dayid'])[0])
    print(debug_graph_fn)
    
    # Step 3
    def detect_outlier(outlier_lst):
        threshold=2
        mean_1 = np.median(outlier_lst)
        std_1 =np.std(outlier_lst)
        
        outliers = [ ]
        for y in outlier_lst:
            z_score= (y - mean_1)/std_1 
            if np.abs(z_score) > threshold:
                outliers.append(y)
        return outliers
    
    # Prepare the previous and next value of chamber 8 (ambient) to figure out the drift
    temp = df[df['chamber'] == 8].groupby('chamber_block').median()
    temp = df[df['chamber'] == 8].groupby('chamber_block').std()
    temp = temp.reset_index()
    temp['to'] = temp['chamber_block'].shift(-1)
    temp['from'] = temp['chamber_block'].shift(1)
    
    outliers = detect_outlier(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol'])
    
    # Plot the drift for diagnostics
    import matplotlib.pyplot as plt
    # CO2
    if(False):
        plt.figure(figsize=(16,8))
        plt.plot(temp['chamber_block'], temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol'], 'black', marker='o')
        #plt.plot(temp.loc[(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']).isin(outliers), 'chamber_block'], temp.loc[(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']).isin(outliers), 'co2.chamber.umol_mol'] - temp.loc[(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']).isin(outliers), 'co2.ambient.umol_mol'], 'red', marker='o', linestyle="None")
        plt.plot(temp.loc[temp['chamber_block'].isin(bad_co2_list), 'chamber_block'], temp.loc[temp['chamber_block'].isin(bad_co2_list), 'co2.chamber.umol_mol'] - temp.loc[temp['chamber_block'].isin(bad_co2_list), 'co2.ambient.umol_mol'], 'red', marker='o', linestyle="None")
        plt.xlabel('Time')
        plt.ylabel('Ch8 CO2')
        plt.axhline(y=np.mean(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']), color='r', linestyle='-')
        plt.axhline(y=np.median(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']), color='b', linestyle='-')
        plt.savefig(project_path_results + debug_graph_fn + '_co2.png', bbox_inches='tight')
        plt.show()
    
    # H2O
    if(False):
        plt.figure(figsize=(16,8))
        plt.plot(temp['chamber_block'], temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol'], 'black', marker='o')
        plt.plot(temp.loc[temp['chamber_block'].isin(bad_h2o_list), 'chamber_block'], temp.loc[temp['chamber_block'].isin(bad_h2o_list), 'h2o.chamber.mmol_mol'] - temp.loc[temp['chamber_block'].isin(bad_h2o_list), 'h2o.ambient.mmol_mol'], 'red', marker='o', linestyle="None")
        plt.xlabel('Time')
        plt.ylabel('Ch8 H2O')
        plt.axhline(y=np.mean(temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol']), color='r', linestyle='-')
        plt.axhline(y=np.median(temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol']), color='b', linestyle='-')
        plt.savefig(project_path_results + debug_graph_fn + '_h2o.png', bbox_inches='tight')
        plt.show()
    
    # Show all points
    if(False):
        temp = df[df['chamber'] == 8]
        plt.figure(figsize=(16,8))
        plt.plot(temp['chamber_block'], temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol'], 'black', marker='o')
        plt.xlabel('Time')
        plt.ylabel('Ch8 CO2')
        plt.axhline(y=np.mean(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']), color='r', linestyle='-')
        plt.axhline(y=np.median(temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']), color='b', linestyle='-')
        plt.savefig(project_path_results + debug_graph_fn2 + '_co2_full.png', bbox_inches='tight')
        plt.show()
    if(False):
        temp = df[df['chamber'] == 8]
        plt.figure(figsize=(16,8))
        plt.plot(temp['chamber_block'], temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol'], 'black', marker='o')
        plt.xlabel('Time')
        plt.ylabel('Ch8 H2O')
        plt.axhline(y=np.mean(temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol']), color='r', linestyle='-')
        plt.axhline(y=np.median(temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol']), color='b', linestyle='-')
        plt.savefig(project_path_results + debug_graph_fn2 + '_h2o_full.png', bbox_inches='tight')
        plt.show()
    
    # H2O
    df.loc[df['chamber_block'].isin(bad_h2o_list), 'h2o.ambient.mmol_mol'] = np.nan
    df.loc[df['chamber_block'].isin(bad_h2o_list), 'h2o.chamber.mmol_mol'] = np.nan
    
    # CO2
    df.loc[df['chamber_block'].isin(bad_co2_list), 'co2.ambient.umol_mol'] = np.nan
    df.loc[df['chamber_block'].isin(bad_co2_list), 'co2.chamber.umol_mol'] = np.nan
    
    return(df)

# Reduction of PAR due to the chamber, correlation by Itay Oz
def PAR_inside(par):
    par_outside = 0.8783 * par + 0.0378
    return(par_outside)

# Correct for the instrument drift in the measured concentrations
# - Uses absolute concentration measurements
def correct_abs_drift(temp):
    # Creat block IDs
    create_block_id(temp)
    
    # Calculate the difference between ambient and sample
    temp['delta_h2o'] = temp['h2o.chamber.mmol_mol'] - temp['h2o.ambient.mmol_mol']
    temp['delta_co2'] = temp['co2.chamber.umol_mol'] - temp['co2.ambient.umol_mol']
    
    # Create correction factor temp, based on the drift
    # Assemble the continuous time series from first to last time in temp
    idx = pd.date_range(start=temp["timestamp"].tolist()[0],
                        end=temp["timestamp"].tolist()[-1],
                        freq="S")

    factors = pd.DataFrame(idx, index=None, columns=["timestamp"])
    factors["h2o_corr_factor"] = np.nan
    factors["co2_corr_factor"] = np.nan

    # Get correction values from chamber 8 data
    grouped = temp[temp["chamber"]==8].groupby("chamber_block")
    for x in grouped:
        h2o_corr = np.median(x[1]["delta_h2o"])
        co2_corr = np.median(x[1]["delta_co2"])
        #x[1]['td'] = pd.to_timedelta(x[1]["timestamp"]).values.astype(np.int64)
        #time = pd.to_timedelta(x[1]['td'].mean()).reset_index()
        #display(np.mean(x[1]["timestamp"]))
        time = pd.Timestamp(np.mean(x[1]["timestamp"])).round('2S')

        # Store the medians at their timestamps in factors df
        factors.loc[factors["timestamp"]==time, "h2o_corr_factor"] = h2o_corr
        factors.loc[factors["timestamp"]==time, "co2_corr_factor"] = co2_corr   

    # Interpolate between datapoints
    factors["h2o_corr_factor"].interpolate(method='linear', limit_direction='both', inplace=True)
    factors["co2_corr_factor"].interpolate(method='linear', limit_direction='both', inplace=True)

    # Join factors into the data by timestamp
    merged = temp.merge(factors, on="timestamp", how="left")
     
    # Apply the correction
    merged["h2o_driftcorr"] = merged["h2o.chamber.mmol_mol"] - merged["h2o_corr_factor"]
    merged["co2_driftcorr"] = merged["co2.chamber.umol_mol"] - merged["co2_corr_factor"]
    
    # Replace original data
    merged["h2o.chamber.mmol_mol"] = merged["h2o_driftcorr"]
    merged["co2.chamber.umol_mol"] = merged["co2_driftcorr"]
    merged.drop('h2o_driftcorr', axis=1, inplace=True)
    merged.drop('co2_driftcorr', axis=1, inplace=True)
    
    # Remove deltas
    merged.drop('delta_h2o', axis=1, inplace=True)
    merged.drop('delta_co2', axis=1, inplace=True)
    
    # Remove correction factors
    merged.drop('h2o_corr_factor', axis=1, inplace=True)
    merged.drop('co2_corr_factor', axis=1, inplace=True)
    
    return(merged)
    
# Remove data from the bad chambers, based on the logbook
def remove_bad_chambers(df_all, chamber_status):
    # Create unique day identifier
    day = df_all['timestamp'].dt.normalize().unique()[0]
    # Load chamber status from the logbook
    temp = chamber_status.loc[chamber_status['timestamp'] == day][chamber_status.columns[3:19]]
    bad_ch_all = temp.columns[(temp == 0).all()].tolist() # The entire chamber measurement is bad
    bad_ch_h2o = temp.columns[(temp == 1).all()].tolist() # H2O is bad, CO2 is ok
    
    # Data from dummy chambers 8 & 16 needs to be kept, but for 16 only until it started measuring
    if(8 in bad_ch_all):
        bad_ch_all.remove(8)     # Used for calibration and ambient data verification
    if(16 in bad_ch_all):
        bad_ch_all.remove(16)    # Used for soil or outside ambient chamber flux values
    if(8 in bad_ch_h2o):
        bad_ch_h2o.remove(8)     # Used for calibration and ambient data verification
    if(16 in bad_ch_h2o):
        bad_ch_h2o.remove(16)    # Used for soil or outside ambient chamber flux values
    
    # Check if the current day is before or after chambers 4 or 16 started measuring a COS chamber,
    # and add to bad chamber list to remove data
    if((4 not in bad_ch_all) & (day > cos_con_start)):
        bad_ch_all.append(4)
    if((16 not in bad_ch_all) & (day > cos_irr_start)):
        bad_ch_all.append(16)
        
    print("    All data bad in chambers: ", bad_ch_all)
    print("    Only H2O bad in chambers: ", bad_ch_h2o)
    # Remove data when entire chamber was bad
    df_all.loc[df_all['chamber'].isin(bad_ch_all),'co2.chamber.umol_mol'] = np.nan
    df_all.loc[df_all['chamber'].isin(bad_ch_all),'flux.co2.umol_m2_s'] = np.nan
    #df_all.loc[df_all['chamber'].isin(bad_ch_all),'temp.irga.c'] = np.nan
    #df_all.loc[df_all['chamber'].isin(bad_ch_all),'press.irga.kpa'] = np.nan
    # Remove data when H2O was bad (and also when all are bad)
    df_all.loc[df_all['chamber'].isin(bad_ch_all + bad_ch_h2o),'h2o.chamber.mmol_mol'] = np.nan
    df_all.loc[df_all['chamber'].isin(bad_ch_all + bad_ch_h2o),'flux.h2o.mmol_m2_s'] = np.nan
    return(df_all)
    
# Removes data when pump flow was bad
def remove_bad_pump_flow(temp):
    # Use a groupby to check the variance and drop entire groups
    create_block_id(temp)
    
    # Prepare standard deviations
    std_df = temp.groupby("chamber_block").std().reset_index()
    std_df.rename(columns={'pump.flow.chamber.lpm':'pump.chamber.std'}, inplace=True)
    std_df.rename(columns={'pump.flow.ambient.lpm':'pump.ambient.std'}, inplace=True)
    std_df = std_df[['chamber_block', 'pump.chamber.std', 'pump.ambient.std']]
    merged = temp.merge(std_df, on="chamber_block", how="left")
    # Remove large SDs
    merged.drop(merged[(merged['pump.chamber.std'] > max_stddev_flow) | (merged['pump.ambient.std'] > max_stddev_flow)].index, axis=0, inplace=True)
    
    # When the flow data doesn't exist
    merged.drop(merged[(merged['pump.flow.chamber.lpm'].isna()) | (merged['pump.flow.ambient.lpm'].isna())].index, axis=0, inplace=True)
    # Pump flow less than 6 LPM
    merged.drop(merged[(merged['pump.flow.chamber.lpm'] < min_abs_flow) | (merged['pump.flow.ambient.lpm'] < min_abs_flow)].index, axis=0, inplace=True)
    # Pump flow not matching
    merged.drop(merged[np.abs(merged['pump.flow.chamber.lpm'] - merged['pump.flow.ambient.lpm']) > max_delta_flow].index, axis=0, inplace=True)
    
    # Remove the index column created mistakenly earlier
    merged.drop('index', axis=1, inplace=True)
    merged.drop('pump.chamber.std', axis=1, inplace=True)
    merged.drop('pump.ambient.std', axis=1, inplace=True)
    
    return merged

# Searches for spikes in the spikes database, and removes them
def remove_spikes(temp):
    temp = temp.copy()
    #temp['mask'] = np.nan # For testing
    # Cycle through the spikes database
    for index, row in spikes.iterrows():
        temp
        # First deal with the Control plot
        if(row['plot'] == 'con'):
            if(row['type'] == 'h2o'):
                # Water spikes
                temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] >= 9) & (temp['chamber'] <= 15),'h2o.chamber.mmol_mol'] = np.nan
                #temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] >= 9) & (temp['chamber'] <= 15),'mask'] = 'ctr h2o' # For testing
            if(row['type'] == 'co2'):
                # CO2 spikes
                temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] >= 9) & (temp['chamber'] <= 15),'co2.chamber.umol_mol'] = np.nan
                #temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] >= 9) & (temp['chamber'] <= 15),'mask'] = 'ctr co2' # For testing
        # Now deal with the irrigation plot
        if(row['plot'] == 'irr'):
            #print('irrigation', row['timestamp'])
            if(row['type'] == 'h2o'):
                # Water spikes
                temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] <= 7),'h2o.chamber.mmol_mol'] = np.nan
                #temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] <= 7),'mask'] = 'irr h2o' # For testing
            if(row['type'] == 'co2'):
                # CO2 spikes
                temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] <= 7),'co2.chamber.umol_mol'] = np.nan
                #temp.loc[(temp['timestamp'] >= row['timestamp']) & (temp['timestamp'] < (row['timestamp'] + pd.Timedelta('+1 hour'))) & (temp['chamber'] <= 7),'mask'] = 'irr co2' # For testing
    
    return(temp)
    
# Calculate fluxes based on flow rate and leaf area
def calculate_flux_chambers(temp, la):
    # Problems: The 2 pumps are not identical in terms of speed. The flux is based on the ambient pump but what about the sample pump?
    
    # The TSI flow meters use SLPM with the following values for air flow:
    air_temperature_tsi_c = 21.11 # [°C]
    air_pressure_tsi_kpa  = 101.3 # [kPa]
    
    # Add leaf area column
    merged = temp.merge(find_leaf_area(day, la), on="chamber", how="left")
    
    # Calculate fluxes with the Jonathan Muller's method
    #merged['flux.h2o.mmol_m2_s'] = 10**3 * calculate_flux(merged['temp.irga.c'], merged['press.irga.kpa']*1000, merged['pump.flow.chamber.lpm'], merged['h2o.ambient.mmol_mol']*10**(-3), merged['h2o.chamber.mmol_mol']*10**(-3), merged['leaf.area.m2'])
    #merged['flux.co2.umol_m2_s'] = 10**6 * calculate_flux(merged['temp.irga.c'], merged['press.irga.kpa']*1000, merged['pump.flow.chamber.lpm'], merged['co2.ambient.umol_mol']*10**(-6), merged['co2.chamber.umol_mol']*10**(-6), merged['leaf.area.m2'])
    
    # Comparable to LI-6400 (cf. manual p. 1-7)
    merged['flux.h2o.mmol_m2_s'] = 10**3 * calculate_h2o_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                     air_pressure_tsi_kpa*1000, #merged['press.irga.kpa']*1000,
                                                                     merged['h2o.ambient.mmol_mol'],
                                                                     merged['h2o.chamber.mmol_mol'],
                                                                     merged['pump.flow.chamber.lpm'],
                                                                     merged['leaf.area.m2'])
    merged['flux.co2.umol_m2_s'] = 10**6 * calculate_co2_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                     air_pressure_tsi_kpa*1000, # merged['press.irga.kpa']*1000,
                                                                     merged['h2o.ambient.mmol_mol'],
                                                                     merged['h2o.chamber.mmol_mol'],
                                                                     merged['co2.ambient.umol_mol'],
                                                                     merged['co2.chamber.umol_mol'],
                                                                     merged['pump.flow.chamber.lpm'],
                                                                     merged['leaf.area.m2'])
    
    # For testing
    #display(merged.loc[merged['chamber'] == 4,['timestamp','chamber','temp.irga.c','press.irga.kpa','h2o.ambient.mmol_mol','h2o.chamber.mmol_mol','co2.ambient.umol_mol','co2.chamber.umol_mol','pump.flow.chamber.lpm','leaf.area.m2']])
    
    # Fluxes with projected, not total LA (Like Yakir & Ori)
    merged['flux.h2o.proj_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                             air_pressure_tsi_kpa*1000, #merged['press.irga.kpa']*1000,
                                                                             merged['h2o.ambient.mmol_mol'],
                                                                             merged['h2o.chamber.mmol_mol'],
                                                                             merged['pump.flow.chamber.lpm'],
                                                                             merged['leaf.area.m2']/proj_la_factor)
    merged['flux.co2.proj_la.umol_m2_s'] = 10**6 * calculate_co2_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                             air_pressure_tsi_kpa*1000, #merged['press.irga.kpa']*1000,
                                                                             merged['h2o.ambient.mmol_mol'],
                                                                             merged['h2o.chamber.mmol_mol'],
                                                                             merged['co2.ambient.umol_mol'],
                                                                             merged['co2.chamber.umol_mol'],
                                                                             merged['pump.flow.chamber.lpm'],
                                                                             merged['leaf.area.m2']/proj_la_factor)
    
    # Fluxes with active LA (reduced from shading)
    merged['flux.h2o.active_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                             air_pressure_tsi_kpa*1000, #merged['press.irga.kpa']*1000,
                                                                             merged['h2o.ambient.mmol_mol'],
                                                                             merged['h2o.chamber.mmol_mol'],
                                                                             merged['pump.flow.chamber.lpm'],
                                                                             merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']))
    merged['flux.co2.active_la.umol_m2_s'] = 10**6 * calculate_co2_flux_li6400(air_temperature_tsi_c, #merged['temp.irga.c'],
                                                                             air_pressure_tsi_kpa*1000, #merged['press.irga.kpa']*1000,
                                                                             merged['h2o.ambient.mmol_mol'],
                                                                             merged['h2o.chamber.mmol_mol'],
                                                                             merged['co2.ambient.umol_mol'],
                                                                             merged['co2.chamber.umol_mol'],
                                                                             merged['pump.flow.chamber.lpm'],
                                                                             merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']))
    
    # Total leaf conductance
    merged['gs.h2o.mol_m2_s'] = calculate_leaf_h2o_conductance(merged['temp.air.ambient.c'],
                                                           merged['temp.air.current.chamber.c'],
                                                           merged['press.irga.kpa']*1000,
                                                           merged['h2o.ambient.mmol_mol'],
                                                           merged['h2o.chamber.mmol_mol'],
                                                           merged['co2.ambient.umol_mol'],
                                                           merged['co2.chamber.umol_mol'],
                                                           merged['pump.flow.chamber.lpm'],
                                                           merged['leaf.area.m2'])
    merged['gs.h2o.proj_la.umol_m2_s'] = calculate_leaf_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/proj_la_factor)
    merged['gs.h2o.active_la.umol_m2_s'] = calculate_leaf_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']))
    # Stomatal conductance
    merged['gs.h2o.stomatal.mol_m2_s'] = calculate_stomatal_h2o_conductance(merged['temp.air.ambient.c'],
                                                           merged['temp.air.current.chamber.c'],
                                                           merged['press.irga.kpa']*1000,
                                                           merged['h2o.ambient.mmol_mol'],
                                                           merged['h2o.chamber.mmol_mol'],
                                                           merged['co2.ambient.umol_mol'],
                                                           merged['co2.chamber.umol_mol'],
                                                           merged['pump.flow.chamber.lpm'],
                                                           merged['leaf.area.m2'])
    merged['gs.h2o.stomatal.proj_la.umol_m2_s'] = calculate_stomatal_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/proj_la_factor)
    merged['gs.h2o.stomatal.active_la.umol_m2_s'] = calculate_stomatal_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']))
    # Boundary layer conductance
    merged['gs.h2o.bl.mol_m2_s'] = calculate_bl_h2o_conductance(merged['temp.air.ambient.c'],
                                                           merged['temp.air.current.chamber.c'],
                                                           merged['press.irga.kpa']*1000,
                                                           merged['h2o.ambient.mmol_mol'],
                                                           merged['h2o.chamber.mmol_mol'],
                                                           merged['co2.ambient.umol_mol'],
                                                           merged['co2.chamber.umol_mol'],
                                                           merged['pump.flow.chamber.lpm'],
                                                           merged['leaf.area.m2'],
                                                           K)
    merged['gs.h2o.bl.proj_la.umol_m2_s'] = calculate_bl_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/proj_la_factor,
                                                                    K)
    merged['gs.h2o.bl.active_la.umol_m2_s'] = calculate_bl_h2o_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']),
                                                                    K)
    
    # Boundary layer conductance
    merged['gs.co2.leaf.mol_m2_s'] = calculate_leaf_co2_conductance(merged['temp.air.ambient.c'],
                                                           merged['temp.air.current.chamber.c'],
                                                           merged['press.irga.kpa']*1000,
                                                           merged['h2o.ambient.mmol_mol'],
                                                           merged['h2o.chamber.mmol_mol'],
                                                           merged['co2.ambient.umol_mol'],
                                                           merged['co2.chamber.umol_mol'],
                                                           merged['pump.flow.chamber.lpm'],
                                                           merged['leaf.area.m2'],
                                                           K)
    merged['gs.co2.leaf.proj_la.umol_m2_s'] = calculate_leaf_co2_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/proj_la_factor,
                                                                    K)
    merged['gs.co2.leaf.active_la.umol_m2_s'] = calculate_leaf_co2_conductance(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']),
                                                                    K)
    
    # Intercellular CO2 conc
     merged['ci.leaf.umol_mol'] = 10**6 * calculate_leaf_co2_concentration(merged['temp.air.ambient.c'],
                                                           merged['temp.air.current.chamber.c'],
                                                           merged['press.irga.kpa']*1000,
                                                           merged['h2o.ambient.mmol_mol'],
                                                           merged['h2o.chamber.mmol_mol'],
                                                           merged['co2.ambient.umol_mol'],
                                                           merged['co2.chamber.umol_mol'],
                                                           merged['pump.flow.chamber.lpm'],
                                                           merged['leaf.area.m2'],
                                                           K)
    merged['ci.leaf.proj_la.umol_mol'] = 10**6 * calculate_leaf_co2_concentration(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/proj_la_factor,
                                                                    K)
    merged['ci.leaf.active_la.umol_mol'] = 10**6 * calculate_leaf_co2_concentration(merged['temp.air.ambient.c'],
                                                                    merged['temp.air.current.chamber.c'],
                                                                    merged['press.irga.kpa']*1000,
                                                                    merged['h2o.ambient.mmol_mol'],
                                                                    merged['h2o.chamber.mmol_mol'],
                                                                    merged['co2.ambient.umol_mol'],
                                                                    merged['co2.chamber.umol_mol'],
                                                                    merged['pump.flow.chamber.lpm'],
                                                                    merged['leaf.area.m2']/(active_la_intercept + active_la_slope*merged['leaf.area.m2']),
                                                                    K)
    
    merged.drop('leaf.area.m2', axis=1, inplace=True)
    return merged
    
# Calculate VPD for each chamber
def calculate_VPD_chambers(temp):
    temp['vpd.air.chamber.kpa'] = calculate_VPD(temp['temp.air.current.chamber.c'], temp['h2o.chamber.mmol_mol'], temp['press.irga.kpa']*1000)/1000
    temp['vpd.air.ambient.kpa'] = calculate_VPD(temp['temp.air.ambient.c'], temp['h2o.ambient.mmol_mol'], temp['press.irga.kpa']*1000)/1000
    return(temp)
    
def calculate_energy_fluxes(temp):
    # Latent heat
    temp['LE.W_m2'] = calculate_latent_heat(temp['temp.air.current.chamber.c'], temp['flux.h2o.mmol_m2_s'], temp['press.irga.kpa']*1000)
    temp['LE.proj_la.W_m2'] = calculate_latent_heat(temp['temp.air.current.chamber.c'], temp['flux.h2o.proj_la.mmol_m2_s'], temp['press.irga.kpa']*1000)
    temp['LE.active_la.W_m2'] = calculate_latent_heat(temp['temp.air.current.chamber.c'], temp['flux.h2o.active_la.mmol_m2_s'], temp['press.irga.kpa']*1000)
    
    # Add leaf area column
    merged = temp.merge(find_leaf_area(day, la), on="chamber", how="left")
    
    # Calculate sensible heat flux H
    # Note!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # Problems of chamber wall temperature: Needs to be corrected after tests in Yatir
    merged['H.W_m2'] = calculate_sensible_heat(merged['temp.air.current.chamber.c'],
                                               merged['temp.air.ambient.c'],
                                               merged['h2o.chamber.mmol_mol'],
                                               merged['pump.flow.chamber.lpm'],
                                               merged['press.irga.kpa']*1000,
                                               merged['leaf.area.m2'])
    
    # Remove soil (or outside ambient chamber) and ambient sensible heat fluxes,
    # they're 0 because the same temperature measurement is used for in and out
    merged.loc[merged['chamber'] == 8,  "H.W_m2"] = np.nan
    merged.loc[merged['chamber'] == 16, "H.W_m2"] = np.nan
    
    # Remove now unnecessary columns
    merged.drop('leaf.area.m2', axis=1, inplace=True)
    return merged

# Writes output files in full temporal resolution
def write_output_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    month_dir = str(date_idx)[0:4] + "-" + str(date_idx)[4:6]
    if(not os.path.exists(out_dir + month_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir + month_dir)
    # Create file name
    out_fn = out_dir + month_dir + "/branch_chambers_" + str(date_idx) + ".csv"
    print(str(len(out_df.dayid)), out_fn) # Shows final file size)
    # organise data for output
    temp_df = out_df
    # Before saving, remove the index
    temp_df.drop('dayid', axis=1, inplace=True)
    temp_df.drop('chamber_block', axis=1, inplace=True)
    # Move timestamp column to the front
    col = temp_df.pop('Counter')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('chamber')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('timestamp')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # Write data
    temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S') # Save file

# Writes output files averaged by plot and hour
def write_1h_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    this_month = str(date_idx)[0:6] #+ "-" + str(date_idx)[4:6]
    if(not os.path.exists(out_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + "/branch_chambers_" + str(this_month) + ".csv"
    # Make 1h means
    temp_df = make_1h_means(out_df)
    # Remove obsolete columns
    temp_df.drop('chamber', axis=1, inplace=True)
    temp_df.drop('Counter', axis=1, inplace=True)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # If the monthly file does not exist, create it with the header
    if not os.path.isfile(out_fn):
        temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')
    else: # otherwise append without writing the header
        temp_df.to_csv(out_fn, mode='a', header=False, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')

# Writes output files averaged by chamber and time
def write_1min_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    this_month = str(date_idx)[0:6] #+ "-" + str(date_idx)[4:6]
    if(not os.path.exists(out_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + "/branch_chambers_" + str(this_month) + ".csv"
    # Make 1h means
    temp_df = make_1min_means(out_df)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # If the monthly file does not exist, create it with the header
    if not os.path.isfile(out_fn):
        temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')
    else: # otherwise append without writing the header
        temp_df.to_csv(out_fn, mode='a', header=False, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')

## Order of corrections and pipeline

In [ ]:
# Applies all the corrections in order. This gets called by the main loops that are in control of reading and writing files
def apply_corrections(temp, date_idx, out_dir):
    if(len(temp['timestamp']) > 240): # Checks if there is enough data to properly do all the corrections
        print("- Applying corrections on ", np.unique(temp['dayid'])[0], "...")
        # Fix column names
        temp = fix_column_names(temp)
        # Fix some values that were moved/renamed etc
        temp = fix_false_values(temp)
        # Every midnight, automatic calibration happened (or not). Remove (it's not real data)
        temp = remove_midnight_4min(temp)
        # For each chamber, the current air temperature
        temp = current_chamber_air_t(temp)
        # Fix PAR, i.e. remove invalid data and make nights 0
        temp = fix_PAR(temp)
        temp = fix_RH(temp)
        # Add Apogee PAR sensor data
        temp = add_apogee_PAR(temp, project_path_par, date_idx)
        # Keep only the last 30s of each measurement period
        #temp = get_last_30s(temp) # Based on timestamps
        temp = keep_last_30s(temp) # Based on timestamps
        # Remove outliers
        temp = remove_ambient_fluctuations(temp)
        # Correct the device drift according to chamber 8
        temp = correct_abs_drift(temp)
        # Remove data from broken chambers, according to the logbook
        temp = remove_bad_chambers(temp, cs)
        # Remove data when pump flow was insufficient or not matching enough
        temp = remove_bad_pump_flow(temp)
        # Despike based on manually identified spikes (Itay)
        temp = remove_spikes(temp)
        # DEBUG MESSAGE
        print("Length: ",len(temp['timestamp']))        
        
        if(len(temp.index) > 10):
            # Correct fluxes given by the datalogger by leaf area
            temp = calculate_flux_chambers(temp, la)
            # Calculate VPD
            temp = calculate_VPD_chambers(temp)
            # Calculate energy fluxes (H & LE)
            temp = calculate_energy_fluxes(temp)
            # Write the data
            write_output_file(temp, date_idx, out_dir + "raw_qc/")
            write_1h_file(temp, date_idx, out_dir + "1h/")
            write_1min_file(temp, date_idx, out_dir + "1min/")
        else:
            print("    No data left to store...")
    else:
        print("- Data of", np.unique(temp['dayid'])[0], "too short for corrections")

In [ ]:
# Run calculations
#-----------------

# List all files in the directory
fn_list = sorted(glob.glob(project_path_chambers + '*/*', recursive=True))
saved = []

# List all PAR files in the directory
par_fn_list = sorted(glob.glob(project_path_par + '*/*', recursive=True))

# Prepare 1h directory. Because files aren't overwritten typically, we need to empty it first
empty_dir(project_path_results + "1h/")
empty_dir(project_path_results + "1min/")

# For all files in the directory
for fn_i, fn in enumerate(fn_list):
    # Only run data in the month list
    current_month = fn.replace(project_path_chambers[:-1], "")[1:8]
    if((current_month in month_list) or (len(month_list) == 0)):
        #display(fn.replace(project_path_chambers[:-1], "")[1:8])
        pass
    else:
        continue
        
    # Debugging message
    if(fn_i % 1 == 0): # % 20 to show every 20th file being loaded
        print( '{:<07}'.format(str(round(fn_i * 100 / len(fn_list), 4))) + "%\t\t" + fn.replace(project_path_chambers[:-1], "")[1:]) # Show status
    
    # Load the current file
    df = read_input_file(fn)
    
    if (fn_i != len(fn_list)-1):
        # Load next file
        df_next = read_input_file_by_lines(fn_list[fn_i+1], lines=1)
        next_day = df_next['dayid'].tolist()[0]
        final_file = False
    else:
        next_day = 0
        final_file = True
  
    # Group by per-day id
    grouped = df.groupby(['dayid'])
    #print("    Days in this file:  ", len(grouped))
    
    # For each group
    for group_i, (this_day, day) in enumerate(grouped):
        
        # If we have saved data and if the day matches append the current
        if (len(saved) > 0):
            if (this_day == saved['dayid'].tolist()[0]):
                day = pd.concat([saved, day], axis=0, ignore_index=True)
                #print("    Current line count: ", str(len(day.dayid)))
        
        #If this is the final group in the file and this is not the final file and the first group of the next file is the same day
        if (group_i == len(grouped)-1) and (not final_file) and (this_day == next_day):
            saved = day
            continue  
        else:
            apply_corrections(day, this_day, project_path_results)
            
print("Done...")